These are the libraries that we are using:

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.preprocessing import PolynomialFeatures

# 1. Decision Tree
## a) Creating highCrime

In [213]:
data_clean=pd.read_csv('C:/Users/Carru/Documents/Trabajos Carlota/ERASMUS/IIT/CS595-Data Science Analytics/Project/Crime Prediction Data/communities-crime-clean.csv')
data_clean.head() #we check that the data is loaded correctly

,state,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,...,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,ViolentCrimesPerPop
0,1,Alabastercity,7,0.01,0.61,0.21,0.83,0.02,0.01,0.41,...,0.03,0.70,0.40,0.34,0.57,0.05,0.06,0.01,0.0,0.06
1,1,AlexanderCitycity,10,0.01,0.41,0.55,0.57,0.01,0.00,0.47,...,0.00,0.93,0.66,0.82,0.84,0.11,0.03,0.01,0.0,0.14
2,1,Annistoncity,3,0.03,0.34,0.86,0.30,0.04,0.01,0.41,...,0.04,0.77,0.59,0.70,0.64,0.06,0.11,0.04,0.0,1.00
3,1,Athenscity,8,0.01,0.38,0.35,0.71,0.04,0.01,0.39,...,0.03,0.78,0.56,0.67,0.71,0.09,0.05,0.00,0.0,0.23
4,1,Auburncity,1,0.04,0.37,0.32,0.70,0.21,0.02,1.00,...,0.12,0.49,0.12,0.00,0.15,0.09,0.09,0.01,0.0,0.15


In [214]:
highCrime=[]
for i in range(len(data_clean)):
    highCrime.append(data_clean.ViolentCrimesPerPop[i] > 0.1)
data_clean['highCrime']=highCrime

#### What are the percentage of positive and negative instances in the dataset?

In [215]:
perc_p= (highCrime.count(True)/len(data_clean))*100
perc_n= (highCrime.count(False)/len(data_clean))*100
print('The percentage of positive is: ', perc_p)
print('The percentage of negative is: ', perc_n)

The percentage of positive is:  62.719518314099346
The percentage of negative is:  37.280481685900654


## b) Decision Tree Classifier

In [216]:
dt = DecisionTreeClassifier(random_state=0)
data=data_clean.drop(['state', 'fold', 'communityname', 'highCrime', 'ViolentCrimesPerPop'],1)
#From the communities-names.txt we see that neither county, community, communityname, state nor fold should be included
#in the model, since they are non-predictive. In the data_clean dataset county and community have already been removed

y=np.asarray(highCrime)
dt.fit(data, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [217]:
y_pred=dt.predict(data)

#### i. What are the training accuracy, precision and recall for this tree?

In [218]:
def applymetrics(y, y_pred):
    acc=accuracy_score(y, y_pred)
    print('The accuracy is: ', acc)
    precision=precision_score(y, y_pred)
    print('The precision is: ', precision)
    recall=recall_score(y, y_pred)
    print('The recall is: ', recall)
applymetrics(y, y_pred)

The accuracy is:  1.0
The precision is:  1.0
The recall is:  1.0


#### ii.	What are the main features used for classification? Can you explain why they make sense (or not)?

In [219]:
df=pd.DataFrame(dt.feature_importances_, data.columns)
df.sort_values(0, ascending=False).head()

,0
PctKids2Par,0.360083
racePctWhite,0.088817
racePctHisp,0.048585
PctLess9thGrade,0.023569
PctEmplManu,0.019933


Applying the Decisision Tree we get that the main features are:
- percentage of kids in family housing with two parents 
- percentage of population that is caucasian
- percentage of population that is of hispanic heritage
- percentage of people 25 and over with less than a 9th grade education
- percentage of people 16 and over who are employed in manufacturing 

The attribute that is given the highest weight is the percentage of kids in family housing with two parents, this makes sense since parents are not going to have a family in a rough area. The rest of the attributes are given a weight close to zero, but it also makes sense to use them as attributes. The higher the population  of hispanic and non-caucasian has historicly lead to a higher crime rate and there is also a higher tendency towards violence when  people are illiterate or have less studies.

## c) Cross-Validation

In [220]:
def applyCross(model, data, y, nfolds):
    print('The accuracy is:', cross_val_score(model, data, y, cv=nfolds, scoring='accuracy').mean())
    print('The precision is: ', cross_val_score(model, data, y, cv=nfolds, scoring='precision').mean())
    print('The recall is: ', cross_val_score(model, data, y, cv=nfolds, scoring='recall').mean())

#### i.	What are the 10-fold cross-validation accuracy, precision, and recall?

In [221]:
applyCross(dt, data, y, 10)

The accuracy is: 0.729020100503
The precision is:  0.789205377179
The recall is:  0.7776


#### ii.	Why are they different from the results in the previous test?
The results in this case are different because the data is divided into training and test. So the model has less observations to practice on and construct a better model. The previous model was clearly overfitting the data.

# 2. Linear Classification
## a) Naïve Bayes
#### i.	What is the 10-fold cross-validation accuracy, precision, and recall for this method?

In [222]:
naive=GaussianNB()
applyCross(naive, data, y, 10)

The accuracy is: 0.761608040201
The precision is:  0.911799814828
The recall is:  0.692


#### ii.	What are the 10 most predictive features? Why do these make sense (or not)? 

In [224]:
naive.fit(data, y)
pred=abs(naive.theta_[0,:]-naive.theta_[1,:])/(naive.sigma_[0,:]+naive.sigma_[1,:])
Naive_features=pd.DataFrame(pred, data.columns)
Naive_features.sort_values(0, ascending=False).head(10)

,0
PctKids2Par,5.001310
FemalePctDiv,4.756886
PctFam2Par,4.593988
pctWInvInc,4.394991
TotalPctDiv,4.385986
PctTeen2Par,3.999354
MalePctDivorce,3.950731
PctYoungKids2Par,3.648301
PctIlleg,3.469889
racePctWhite,3.443228


The most important attributes are the following: 
- percentage of kinds in family housing with two parents
- percentage of females who are divorced
- percentage of families (with kids) that are headed by two parents 
- percentage of households with investment / rent income in 1989 
- percentage of population who are divorced
- percentage of kids age 12-17 in two parent households
- percentage of males who are divorced
- percent of kids 4 and under in two parent households
- percentage of kids born to never married 
- percentage of population that is caucasian 

In this case, the parameters have a higher weight, also they all seem to be egatively correlated with highCrime (ie. the higher their percentage, the lower the crime in that area).

#### iii.	How do these results compare with your results from decision trees, above?
Comparing the case where we applied cross validation in the decision tree, we observe that the accuracy in this case is higher which means that the model predicts better which attributes lead to a high crime. The recall is lower, this means that the model incorrectly classifies more observations as false than the decision tree.

## b) Support Vector Machine (SVM)

#### i.	 What is the 10-fold cross-validation accuracy, precision, and recall for this method?

In [225]:
svm=LinearSVC(random_state=0) 
applyCross(svm, data, y, 10)

The accuracy is: 0.796233668342
The precision is:  0.845404856531
The recall is:  0.8344


#### ii.	What are the 10 most predictive features? Why do these make sense (or not)?

In [226]:
svm.fit(data, y)
SVM_features=pd.DataFrame(abs(svm.coef_)[0], data.columns)
SVM_features.sort_values(0, ascending=False).head(10)

,0
pctWInvInc,1.888488
PersPerOccupHous,1.755123
racePctWhite,1.500218
PctKids2Par,1.190327
RentHighQ,1.066884
MalePctDivorce,1.065697
NumUnderPov,1.051548
NumStreet,1.019154
PctOccupMgmtProf,1.014674
population,1.002300


The most important attributes in this case are: 
- percentage of households with investment / rent income in 1989 
- mean persons per household 
- percentage of population that is caucasian
- percentage of kinds in family housing with two parents
- rental housing - upper quartile rent
- percentage of male who are divorced
- number of people under the poverty level
- number of homeless people counted in the street 
- percentage of people 16 and over who are employed in management or professional occupations 
- population for community

In this case we can see that some of these features historicly have been highly correlated with high crime rate, an example of this could be the "number of people under the poverty level and number of homeless people counted in the street". 

#### iii.	How do these results compare with your results from decision trees, above?
The accuracy is higher than in the previous cases, this makes sense because the model is using features that make more sense to a higher crime rate. Also the recall and the precision are high and their value is more or less the same. This means that from the models seen so far, this is the one that misclassifies the least (in terms of number of false negative)

# 3. Regression
## a) Linear Regression
#### i.	Using 10-fold cross-validation, what is the estimated mean-squared-error (MSE) of the model?

In [26]:
y2=data_clean['ViolentCrimesPerPop']
lm=LinearRegression()
mse1=-cross_val_score(lm, data, y2, cv=10, scoring='neg_mean_squared_error').mean() 
mse1

0.020093969304445312

#### ii.	What is the MSE on the training set (train on all the data then test on it all)?

In [27]:
lm.fit(data,y2)
y_pred2=lm.predict(data)
mse=mean_squared_error(y2, y_pred2)
print('The MSE is:', mse)

The MSE is: 0.0165167748803


In this case we get a lower error, this makes sense since the model had more data to train on

#### iii. What features are most predictive of a high crime rate? A low crime rate?

In [28]:
LRegression_features=pd.DataFrame(lm.coef_, data.columns)
Most_p=LRegression_features.sort_values(0, ascending=False).head()
Least_p=LRegression_features.sort_values(0).head()
Most_p #Most predictive features

,0
PersPerOccupHous,0.635088
PctHousOwnOcc,0.568133
MalePctDivorce,0.458517
PctRecImmig8,0.432511
MedRent,0.372728


In [29]:
Least_p #Least predictive features

,0
PctPersOwnOccup,-0.675694
TotalPctDiv,-0.561924
whitePerCap,-0.351016
PctKids2Par,-0.322651
OwnOccLowQuart,-0.308170


## b) Ridge Regression
#### i.	What is the estimated MSE of the model under 10-fold CV?

In [30]:
ridge=RidgeCV(alphas=(10, 1, 0.1, 0.01, 0.001))
mseR=-cross_val_score(ridge, data, y2, cv=10, scoring='neg_mean_squared_error').mean() 
mseR

0.019795021348158991

#### ii.	What is the MSE on the training set (train on all the data then test on it all)?

In [31]:
ridge.fit(data, y2)
ridge_pred=ridge.predict(data)
mseRp=mean_squared_error(y2, y_pred2)
print('The MSE is:', mseRp)

The MSE is: 0.0165167748803


#### iii.	What is the best alpha?

In [32]:
ridge.alpha_

1.0

#### i.	What does this say about the amount of overfitting in linear regression for this problem?

Since alpha can take a value from 0 to infinity, we can say that 1 means there is not much overfitting

## c) Quadratic Polynomial Regression
#### i.	What is the estimated MSE of the model under 10-fold CV?

In [38]:
poly=PolynomialFeatures(degree=2)
new_data=poly.fit_transform(data)
-cross_val_score(lm, new_data, y2, cv=10, scoring='neg_mean_squared_error').mean()

0.12990008054870189

#### ii. What is the MSE on the training set (train on all the data then test on it all)?

In [39]:
lm.fit(new_data, y2)
pol_pred=lm.predict(new_data)
mse=mean_squared_error(y2, pol_pred)
print('The MSE is:', mse)

The MSE is: 9.18259450887e-29


#### iii.	Does this mean the quadratic model is better than the linear model for this problem?
No, the quadratic model is probably overfitting. The mse that we obtain with cross validation is more accurate since the model is testing on data that it hasn't seen. If we look at the mean squared error in this case, we observe that the value is higher than in the case of linear regression

# 4. Dirty Data
### a.	Are the CV results better or worse? What does this say about the effect of missing values?

In [197]:
data_full=pd.read_csv('~/Documents/Trabajos Carlota/ERASMUS/IIT/CS595-Data Science Analytics/Project/Crime Prediction Data/communities-crime-full.csv',na_values='?')
data_full.shape

(1994, 128)

The number of rows is the same but there are more columns in this dataset

In [198]:
null_columns=data_full.columns[data_full.isnull().any()]
null_columns #columns with null attributes

Index(['county', 'community', 'OtherPerCap', 'LemasSwornFT', 'LemasSwFTPerPop',
       'LemasSwFTFieldOps', 'LemasSwFTFieldPerPop', 'LemasTotalReq',
       'LemasTotReqPerPop', 'PolicReqPerOffic', 'PolicPerPop',
       'RacialMatchCommPol', 'PctPolicWhite', 'PctPolicBlack', 'PctPolicHisp',
       'PctPolicAsian', 'PctPolicMinor', 'OfficAssgnDrugUnits',
       'NumKindsDrugsSeiz', 'PolicAveOTWorked', 'PolicCars', 'PolicOperBudg',
       'LemasPctPolicOnPatr', 'LemasGangUnitDeploy', 'PolicBudgPerPop'],
      dtype='object')

In [199]:
for i in range(len(null_columns)):
    data_full[null_columns[i]].fillna(data_full[null_columns[i]].mean(), inplace=True)


In [202]:
data_full=data_full.drop(['state', 'county', 'community', 'fold', 'communityname'],1)
#We have to create the variable highCrime again:
data_full['highCrime']=(data_full.ViolentCrimesPerPop>0.1)
y=np.asarray(data_full['highCrime'])
data_full=data_full.drop(['highCrime', 'ViolentCrimesPerPop'], 1)


In [205]:
#First we use all the data for training and testing
dt.fit(data_full, y)
y_pred=dt.predict(data_full)
applymetrics(y, y_pred)

The accuracy is:  1.0
The precision is:  1.0
The recall is:  1.0


In [207]:
df=pd.DataFrame(dt.feature_importances_, data_full.columns)
df.sort_values(0, ascending=False).head()

,0
PctKids2Par,0.362240
racePctWhite,0.089055
racePctHisp,0.050001
PctLess9thGrade,0.020450
MedRent,0.019161


In [206]:
applyCross(dt, data_full, y, 10)

The accuracy is: 0.761765331633
The precision is:  0.813537877163
The recall is:  0.80493968254


The results of Cross Validation are better than those obtained previously, the reason for this is that we substituted the missing values for a the average of each column. Even though these aren't the actual results, the rest of the values of the attributes would provide more information so that we can create a better model.